# While Loop with Jobflow

This notebook demonstrates using the WhileNode with Jobflow backend.

## Setup

In [ ]:
import sys
from pathlib import Path

# Add source to path
src_path = Path.cwd().parent / "python_workflow_definition" / "src"
sys.path.insert(0, str(src_path))

# Add workflow examples to path so they can be imported
workflow_examples_path = Path.cwd().parent / "example_workflows" / "while_loop"
sys.path.insert(0, str(workflow_examples_path))

from jobflow import run_locally

## Example 1: Load While Loop from JSON

Load a simple counter workflow with while loop from JSON.

In [ ]:
from python_workflow_definition.jobflow import load_workflow_json

# Create the JSON workflow file path
json_file = Path.cwd().parent / "example_workflows" / "while_loop" / "simple_counter.json"

# Load the workflow
flow = load_workflow_json(str(json_file))

print(f"Loaded Flow with {len(flow.jobs)} jobs")
print(f"Job names: {[job.name for job in flow.jobs]}")

## Example 2: Run the Flow

Execute the while loop workflow locally.

In [ ]:
# Run the flow locally
responses = run_locally(flow)

print("Flow completed!")
print(f"Number of responses: {len(responses)}")

# Get the final result
for response in responses.values():
    print(f"Result: {response[1].output}")

## Example 3: Expression-based Condition

Load and run a while loop with expression-based condition.

In [ ]:
json_file_expr = Path.cwd().parent / "example_workflows" / "while_loop" / "simple_counter_expression.json"

flow_expr = load_workflow_json(str(json_file_expr))
responses_expr = run_locally(flow_expr)

print("Expression-based flow completed!")
for response in responses_expr.values():
    print(f"Result: {response[1].output}")

## Example 4: Create While Loop Manually

Create a while loop workflow using Python API and export to JSON.

In [ ]:
from python_workflow_definition.models import (
    PythonWorkflowDefinitionWorkflow,
    PythonWorkflowDefinitionWhileNode,
    PythonWorkflowDefinitionInputNode,
    PythonWorkflowDefinitionOutputNode,
    PythonWorkflowDefinitionEdge,
)

# Define workflow with custom parameters
workflow = PythonWorkflowDefinitionWorkflow(
    version="0.1.0",
    nodes=[
        PythonWorkflowDefinitionInputNode(id=0, type="input", name="n", value=20),
        PythonWorkflowDefinitionInputNode(id=1, type="input", name="m", value=0),
        PythonWorkflowDefinitionWhileNode(
            id=2,
            type="while",
            conditionFunction="workflow.is_less_than",
            bodyFunction="workflow.increment_m",
            maxIterations=500,
        ),
        PythonWorkflowDefinitionOutputNode(id=3, type="output", name="result"),
    ],
    edges=[
        PythonWorkflowDefinitionEdge(source=0, target=2, targetPort="n"),
        PythonWorkflowDefinitionEdge(source=1, target=2, targetPort="m"),
        PythonWorkflowDefinitionEdge(source=2, sourcePort="m", target=3),
    ],
)

# Save to JSON
output_file = Path("/tmp/custom_while_loop_jobflow.json")
workflow.dump_json_file(output_file)

print(f"Saved workflow to {output_file}")
print(workflow.dump_json())

## Example 5: Load and Execute Custom Workflow

In [ ]:
# Load and run the custom workflow
flow_custom = load_workflow_json(str(output_file))
responses_custom = run_locally(flow_custom)

print("Custom workflow completed!")
for response in responses_custom.values():
    result = response[1].output
    print(f"Final result: {result}")
    if isinstance(result, dict) and "m" in result:
        print(f"Counter reached: {result['m']}")

## Example 6: Fibonacci with Accumulator

More complex example using accumulator pattern.

In [ ]:
# Create Fibonacci workflow
fib_workflow = PythonWorkflowDefinitionWorkflow(
    version="0.1.0",
    nodes=[
        PythonWorkflowDefinitionInputNode(id=0, type="input", name="n", value=10),
        PythonWorkflowDefinitionInputNode(id=1, type="input", name="current", value=0),
        PythonWorkflowDefinitionInputNode(id=2, type="input", name="a", value=0),
        PythonWorkflowDefinitionInputNode(id=3, type="input", name="b", value=1),
        PythonWorkflowDefinitionInputNode(id=4, type="input", name="results", value=[]),
        PythonWorkflowDefinitionWhileNode(
            id=5,
            type="while",
            conditionExpression="current < n",
            bodyFunction="nested_workflow.fibonacci_step",  # You'd need to define this
            maxIterations=50,
        ),
        PythonWorkflowDefinitionOutputNode(id=6, type="output", name="fibonacci_result"),
    ],
    edges=[
        PythonWorkflowDefinitionEdge(source=0, target=5, targetPort="n"),
        PythonWorkflowDefinitionEdge(source=1, target=5, targetPort="current"),
        PythonWorkflowDefinitionEdge(source=2, target=5, targetPort="a"),
        PythonWorkflowDefinitionEdge(source=3, target=5, targetPort="b"),
        PythonWorkflowDefinitionEdge(source=4, target=5, targetPort="results"),
        PythonWorkflowDefinitionEdge(source=5, target=6),
    ],
)

print("Fibonacci workflow created (note: requires fibonacci_step function to be defined)")

## Summary

This notebook demonstrated:
1. Loading while loop workflows from JSON into Jobflow
2. Running workflows locally with `run_locally()`
3. Using both function-based and expression-based conditions
4. Creating workflows programmatically with the Python API
5. Exporting workflows to JSON format
6. More complex workflows with accumulator patterns